#**Graded Challenge 3**

##**Identity**

Ihsan Aulia Rahman

Full Time Data Science

HCK-Batch-001

#**A. Description**

##**A. 1. Problem Statement**

The global fashion e-commerce platform The Look is looking to evaluate it's sales report. This evaluation is concering particularly about user retention. We are tasked to find informations around retentive users.

As for who categorizes as retentive user, we are setting the standard as a user who made 3 transactions or more in the first semester of year 2022. From this definition we are venturing to seek the answers of questions surrounding retentive users, such as:

1. Who are the retentive users in our user database?
2. How many transactions do retentive users made?
3. How much money do retentive users spent?
4. Where do retentive users come from?
5. What correlating factor(s) that could influence user retention?

##**A.2. Objective**

The goals of this assignment are to answer questions from the problem statement using queries into company data, present the discoveries in a clear and concise manner, and to infer all the informations with analyses and conclusion.

#**B. Libraries**

Here we import the necessary function libraries for this assignment.

In [1]:
!pip install mysql-connector-python

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import scipy.stats as stats
import sympy as sy
import mysql.connector

from numpy.linalg import inv, det, eig
from numpy import diag
from pylab import *
from sympy.tensor.array import derive_by_array
from google.colab import auth
from google.cloud import bigquery

def plot_vector2d(vector2d, origin=[0, 0], **options):
    return plt.arrow(origin[0], origin[1], vector2d[0], vector2d[1],
              head_width=0.2, head_length=0.3, length_includes_head=True,
              **options)

In [ ]:
auth.authenticate_user()
client = bigquery.Client(project = 'week3-360509')

#**C. Problem Elaborations**

##**Codes**

Here are the codes to query the needed tables straight from BigQuery.

In [ ]:
Retention_Rate = client.query(
'''
SELECT DISTINCT
#Querying number of retentive users
(
 SELECT COUNT(*) AS Retentive_Users #Counting rows of retentive users table
 FROM(
      SELECT COUNT(*) AS Number_of_Transactions #Querying retentive users
      FROM bigquery-public-data.thelook_ecommerce.orders
      WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
      GROUP BY user_id
      HAVING COUNT(*)>=3
      ORDER BY COUNT(*) DESC
     )
) AS Loyal_Users,

#Querying total number of users
(
 SELECT COUNT(*) AS Total_Users
 FROM bigquery-public-data.thelook_ecommerce.users
) AS Total_Users,

#Querying the percentage of retentive users to total number of users
(
 (
 SELECT COUNT(*) AS Retentive_Users
 FROM(
      SELECT COUNT(*) AS Number_of_Transactions
      FROM bigquery-public-data.thelook_ecommerce.orders
      WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
      GROUP BY user_id
      HAVING COUNT(*)>=3
      ORDER BY COUNT(*) DESC
     )
 )
 /
 (
 SELECT COUNT(*) AS Total_Users
 FROM bigquery-public-data.thelook_ecommerce.users
 )
 *100
) AS Percentage_of_Retentive_Users

FROM bigquery-public-data.thelook_ecommerce.orders 
LEFT JOIN bigquery-public-data.thelook_ecommerce.users
ON user_id = id
'''
).to_dataframe()

In [ ]:
Retentive_User_Stats = client.query(
'''
#Querying transaction statistics of retentive users
SELECT 
MIN(Number_of_Transactions) AS Lowest_Number_of_Transactions,
MAX(Number_of_Transactions) AS Highest_Number_of_Transactions,
AVG(Number_of_Transactions) AS Average_Number_of_Transactions

FROM(
     SELECT 
     user_id AS User_ID,
     COUNT(*) AS Number_of_Transactions

     FROM bigquery-public-data.thelook_ecommerce.orders
     WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
     GROUP BY user_id
     HAVING COUNT(*)>=3
     ORDER BY COUNT(*) DESC
    )
'''
).to_dataframe()

In [ ]:
Retentive_User_Spending = client.query(
'''
#Querying spending statistics of retentive users
SELECT 
MIN(Total_Spending) AS Lowest_Spending,
MAX(Total_Spending) AS Highest_Spending,
AVG(Total_Spending) AS Average_Spending

FROM(
     SELECT #Querying retentive users table
     user_id AS User_ID_Orders,
     COUNT(*) AS Number_of_Transactions
     
     FROM bigquery-public-data.thelook_ecommerce.orders
     WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
     GROUP BY user_id
     HAVING COUNT(*)>=3
     ORDER BY COUNT(*) DESC
    )
JOIN(
     SELECT #Querying spending amount of all users
     user_id as User_ID_Items,
     SUM(sale_price) AS Total_Spending
     
     FROM bigquery-public-data.thelook_ecommerce.order_items
     WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
     GROUP BY user_id
     ORDER BY SUM(sale_price) ASC
     )
ON User_ID_Orders = User_ID_Items

'''
).to_dataframe()

In [ ]:
Retentive_User_Source = client.query(
'''
#Querying number of retentive users by traffic sources
SELECT 
Traffic_Source AS Traffic_Source,
COUNT (*) AS Numbers

FROM(
     SELECT #Querying retentive users table
     user_id AS User_ID_Orders,
     COUNT(*) AS Number_of_Transactions
     
     FROM bigquery-public-data.thelook_ecommerce.orders
     WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
     GROUP BY user_id
     HAVING COUNT(*)>=3
     ORDER BY COUNT(*) DESC
    )
JOIN(
     SELECT #Querying traffic sources of all users
     id AS User_ID_Users,
     traffic_source AS Traffic_Source
     
     FROM bigquery-public-data.thelook_ecommerce.users
     )
ON User_ID_Orders = User_ID_Users
GROUP BY Traffic_Source
ORDER BY Numbers DESC

'''
).to_dataframe()

In [ ]:
Source_and_Retention = client.query(
'''
#Querying traffic sources and retention of all users
SELECT 
Traffic_Source AS Traffic_Source,
IFNULL(User_Retention, 'Intermittent') AS User_Retention

FROM(
     SELECT #Querying retention of users in retention period
     user_id as User_ID_Orders, 
     COUNT(order_id) AS Numbers_of_Transactions, 
     CASE WHEN COUNT(order_id) >= 3  THEN 'Retentive'
     ELSE 'Intermittent' END AS User_Retention
     FROM bigquery-public-data.thelook_ecommerce.orders
     WHERE created_at > "2022-01-01" and created_at < "2022-06-30"
     GROUP BY user_id 
    )
RIGHT JOIN(
     SELECT #Querying traffic sources of all users
     id AS User_ID_Users,
     traffic_source AS Traffic_Source
     
     FROM bigquery-public-data.thelook_ecommerce.users
     )
ON User_ID_Orders = User_ID_Users
ORDER BY User_ID_Users ASC
'''
).to_dataframe()

##**C.1. How much is the percentage of high retention users to all users?**

To find out how many of our users who are categorized as retentive, let's open the queried table.

In [ ]:
Retention_Rate

,Loyal_Users,Total_Users,Percentage_of_Retentive_Users
0,1041,100000,1.041


There are only a bit more than a thousand users considered retentive out of one hundred thousand users in our database. Retentive users consist only 1 percent of our user base, making them a rarity.

##**C.2. How much is the highest, lowest and average number of transactions of retentive users?**

Now we know that retentive users are not common among our user base, but just how many purchases do they make?

In [ ]:
Retentive_User_Stats

,Lowest_Number_of_Transactions,Highest_Number_of_Transactions,Average_Number_of_Transactions
0,3,4,3.161383


Retentive users only made 3 or 4 purchases within first semester of 2022. With the average number falls closer to 3 than 4, it seems most retentive users only made 3 purchases in the time period.

##**C.3. How much is the highest, lowest and average spending of retentive users?**

And how much do they spend during the time period?

In [ ]:
Retentive_User_Spending

,Lowest_Spending,Highest_Spending,Average_Spending
0,45.929999,1156.889996,272.147215


While one of the retentive users spent more than 1000 dollars during the time period, another spent only a little below 46 dollars. The average spending amount leans towards the latter at 272 dollars.

##**C.4. From what traffic sources do retentive users come?**

Next, we want to find out where do these prized users found their way to make purchase in our company.

In [ ]:
Retentive_User_Source

,Traffic_Source,Numbers
0,Search,739
1,Organic,136
2,Facebook,67
3,Email,55
4,Display,44


Almost all of our retentive users made purchase through specific search. Organic search came on second at only one-tenth of all of them while the rest is divided fairly even between other three sources.

##**C.5. Is there a correlation between traffic sources and user retention?**

The findings in traffic source table helped us refine our question as to whether a factor could influence user retention. We are going to test if trffic source do correlates with user retention by using the Chi-square method. 

Let's begin with establishing hypotheses:

H0: There are correlation between user traffic source and retention. Marked with P value equal or less than 0.05

H1: There are no correlation between user traffic source and retention. Marked with P value more than 0.05

In [ ]:
Source_and_Retention

,Traffic_Source,User_Retention
0,Organic,Intermittent
1,Search,Intermittent
2,Search,Intermittent
3,Search,Intermittent
4,Search,Intermittent
...,...,...
99995,Search,Intermittent
99996,Display,Intermittent
99997,Facebook,Intermittent
99998,Search,Intermittent


In [ ]:
Chi_Table=pd.crosstab(Source_and_Retention['Traffic_Source'],Source_and_Retention['User_Retention'])
stat, P, dof, expected = stats.chi2_contingency(Chi_Table)
print('P value = %.3f' % (P))
if P <= 0.05:
    print('User traffic sources and retention rate are probably related.')
else:
    print('User traffic sources and retention rate are probably unrelated.')

P Value = 0.546
User traffic sources and retention rate are probably unrelated.


With P value well above the threshold, we reject null hypothesis (H0). It seems from the test that user traffic sources and retention rate are not related with each other.

#**D. Verdicts**

##**Analyses**

We have succesfully answered questions in the problem statements. The queries provided clear measures as to what is, who are, how much, when and where do retentive users come from and made purchase from our company. We set the standard of retentive user as a user who made 3 or more transactions within the first semester of year 2022 and from there we have measured some metrics on them.

First, we found out that retentive users are considered rare at only around 1% of total users. While rare, they don't spend much or more often than intermittent users.

Then, we used traffic source as subject in our test to see if it correlates to user retention. While retentive users show very significant preference to come by search, the test concluded that there is no strong correlation between that and user retention, which could mean that the trait is also shared by intermittent users. 



##**Assumptions**

By those analyses, we could assume that there's no actual "retention" in our user base by the established standard. Those who are considered retentive aren't that much different than others who aren't. Our main assumption is that individual users don't purchase fashion items so often like other commodities. 

With these we could better focus our effort in expanding our user base or marketing high value purchases instead of catering to user retention. We could increase our conversion funnel with marketing and direct our user base towards more profitable items.

##**Conclusion**

We wrap this assignment by saying that the pursue of user retention is would likely be fruitless than widening and strengthening our user base.